In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
from keras.models import Sequential
import keras
from keras.layers import Dense, InputLayer


In [2]:
#pip install tensorflow==2.12.0

In [3]:
data=pd.read_csv(r"final_ipl_data1.csv")

In [66]:
data

,BattingTeam,BowlingTeam,runs_left,balls_left,wickets_remaining,target,crr,rrr,results
0,Gujarat Titans,Rajasthan Royals,131.0,119,10,131.0,0.00,6.61,1
1,Gujarat Titans,Rajasthan Royals,131.0,118,10,131.0,0.00,6.66,1
2,Gujarat Titans,Rajasthan Royals,130.0,117,10,131.0,2.00,6.67,1
3,Gujarat Titans,Rajasthan Royals,130.0,116,10,131.0,1.50,6.72,1
4,Gujarat Titans,Rajasthan Royals,130.0,115,10,131.0,1.20,6.78,1
...,...,...,...,...,...,...,...,...,...
85934,Royal Challengers Bangalore,Kolkata Knight Riders,143.0,31,1,223.0,5.39,27.68,0
85935,Royal Challengers Bangalore,Kolkata Knight Riders,142.0,30,1,223.0,5.40,28.40,0
85936,Royal Challengers Bangalore,Kolkata Knight Riders,142.0,29,1,223.0,5.34,29.38,0
85937,Royal Challengers Bangalore,Kolkata Knight Riders,141.0,29,1,223.0,5.41,29.17,0


In [4]:
#Extracting feature variables and class variables
fv=data.iloc[:,:-1]
cv=data.iloc[:,-1]

In [5]:
# Splitting tha data to train and test splits
x_train,x_test,y_train,y_test = train_test_split(fv,cv,test_size=0.2, random_state=42, stratify=cv)


In [6]:
#Dividing te dataset based on the type if the variables
numerical_data=x_train.select_dtypes(include=["int64","float64"])
cat_data=x_train.select_dtypes(include=["object"])

In [19]:
#Pipeline to impute and scale numerical columns
num_p=Pipeline([("imputing_n",SimpleImputer()),("scaling",StandardScaler())])
#Pipeline to impute and Encode nominal columns
cp=Pipeline([("imputing_c",SimpleImputer(strategy="most_frequent")),("Encoder",OneHotEncoder())])

In [20]:
#Pipeline for column transformation to apply for different types of data
ct=ColumnTransformer([("nominal",cp,cat_data.columns),("numerical",num_p,numerical_data.columns)],remainder="passthrough")

In [21]:
ct

ColumnTransformer(remainder='passthrough',
                  transformers=[('nominal',
                                 Pipeline(steps=[('imputing_c',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('Encoder', OneHotEncoder())]),
                                 Index(['BattingTeam', 'BowlingTeam'], dtype='object')),
                                ('numerical',
                                 Pipeline(steps=[('imputing_n',
                                                  SimpleImputer()),
                                                 ('scaling',
                                                  StandardScaler())]),
                                 Index(['runs_left', 'balls_left', 'wickets_remaining', 'target', 'crr', 'rrr'], dtype='object'))])

In [22]:
x_test.shape

(17188, 8)

In [23]:
x_train_pp=ct.fit_transform(x_train)
x_test_pp=ct.transform(x_test)

In [24]:
print(x_train_pp.shape)
print(x_test_pp.shape)
print(y_train.shape)
print(y_test.shape)

(68751, 24)
(17188, 24)
(68751,)
(17188,)


In [40]:
from keras.utils import to_categorical

# Assuming y_train is a 1D array containing class labels
y_train_encoded = to_categorical(y_train,num_classes=2)

# Assuming y_test is a 1D array containing class labels
y_test_encoded = to_categorical(y_test,num_classes=2)

In [14]:
from keras.models import Model
from keras.layers import Input, Dense

def create_ann_model(input_shape):
    # Input layer
    inputs = Input(shape=(input_shape,))
    
    # Dense hidden layers
    dense1 = Dense(64, activation='relu')(inputs)
    dense2 = Dense(32, activation='relu')(dense1)
    
    # Output layers for batting team and bowling team
    output_batting = Dense(1, activation='sigmoid', name='output_batting')(dense2)
    output_bowling = Dense(1, activation='sigmoid', name='output_bowling')(dense2)
    
    # Define the model with multiple outputs
    model = Model(inputs=inputs, outputs=[output_batting, output_bowling])
    
    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Example usage:
input_shape = x_train_pp.shape[1]
ann_model = create_ann_model(input_shape)


In [47]:
input_shape = x_train_pp.shape[1]
model=Sequential()
model.add(InputLayer(input_shape=(input_shape,)))
model.add(Dense(5,activation="relu",use_bias=True))
model.add(Dense(4,activation="relu",use_bias=True))
model.add(Dense(2,activation="softmax",use_bias=True))

In [48]:
model.compile(optimizer='adam',loss="binary_crossentropy",metrics=["accuracy"])

In [56]:
from keras.callbacks import ModelCheckpoint

# Define the filepath where you want to save the best model weights
checkpoint_filepath = r'C:\Users\LENOVO\innomatics_python\Deep_Learning\best_ipl_model_weights.keras'

# Create a ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',  # or 'val_accuracy' depending on your preference
    save_best_only=True,
    mode='min'  # or 'max' if monitoring accuracy
)

# Train the model with the ModelCheckpoint callback
history = model.fit(
    x_train_pp, y_train_encoded,
    epochs=50,
    batch_size=100,
    validation_data=(x_test_pp, y_test_encoded),
    callbacks=[checkpoint_callback]  # Pass the checkpoint callback here
)


Epoch 1/50
688/688 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8244 - loss: 0.3620 - val_accuracy: 0.8229 - val_loss: 0.3652
Epoch 2/50
688/688 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8248 - loss: 0.3602 - val_accuracy: 0.8236 - val_loss: 0.3656
Epoch 3/50
688/688 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8258 - loss: 0.3587 - val_accuracy: 0.8237 - val_loss: 0.3654
Epoch 4/50
688/688 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8252 - loss: 0.3596 - val_accuracy: 0.8257 - val_loss: 0.3644
Epoch 5/50
688/688 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8254 - loss: 0.3588 - val_accuracy: 0.8237 - val_loss: 0.3644
Epoch 6/50
688/688 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8267 - loss: 0.3579 - val_accuracy: 0.8234 - val_loss: 0.3650
Epoch 7/50
688/688 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8284 - loss: 0.3547 - val_accuracy: 0.8246 - val_loss: 0.3644
Epoch 8/50
688/688 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8264 - loss: 0.3581 - val_accuracy: 0.

In [57]:
from keras.models import load_model

# Load the model architecture
best_model = load_model(r'C:\Users\LENOVO\innomatics_python\Deep_Learning\best_ipl_model_weights.keras')

In [69]:
new_data_point = x_test_pp[0].reshape(1, -1)
probability= best_model.predict(new_data_point)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


In [70]:
np.argmax(best_model.predict(new_data_point))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


0

In [59]:
probability

array([[9.999924e-01, 7.673831e-06]], dtype=float32)

In [61]:
predictions = np.array([[9.999924e-01, 7.673831e-06]])

# Convert probabilities to percentages
probabilities_percentage = predictions * 100
probabilities_percentage

array([[9.999924e+01, 7.673831e-04]])

In [71]:
input_data = pd.DataFrame({
    'BattingTeam': ["Chennai Super Kings"],
    'BowlingTeam': ["Delhi Capitals"],
    'runs_left': [10],
    'balls_left': [20],
    'wickets_remaining': [5],
    'target': [200],
    'crr': [4.6],
    'rrr': [5.6]
})

# Transform the input data using the ColumnTransformer
x = ct.transform(input_data)

In [72]:
new_data_point = x.reshape(1, -1)
probabilities = best_model.predict(new_data_point)
print(probabilities)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
[[0.07313694 0.92686313]]


In [73]:
np.argmax(best_model.predict(new_data_point))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


1

In [76]:
probabilities[0][0]*100

7.313694059848785